In [22]:
import pandas as pd
import pydeck as pdk
from ODPworkspace_demotools import mapOQS, pydeck_plot
import matplotlib.pyplot as plt
from shapely.geometry import shape, MultiPolygon, Polygon
from shapely import wkt
from time import time
import geopandas as gpd
import os
from rasterio.io import MemoryFile
from IPython.display import display
from ipywidgets import widgets
from IPython.display import clear_output
import base64
import io
import warnings

In [2]:
from odp.client import OdpClient
from odp.dto import Metadata
from odp.dto.catalog import DatasetDto, DatasetSpec
from odp.dto.common.contact_info import ContactInfo

In [3]:
client = OdpClient()

In [4]:
dataset = client.catalog.get(("8131a076-af24-483b-b9a1-20a37433ef46")) ## UUID of ProtectedSeas dataset from the catalog
dataset.metadata.display_name

'ProtectedSeas MPA Dataset'

### List the Table Schema of ProtectedSeas Database

In [6]:
PSmetadata = client.tabular.get_schema(dataset)

In [7]:
table_schema = PSmetadata.table_schema

schema_data = []
for column, details in table_schema.items():
    schema_data.append({
        "Column Name": column,
        "Type": details.get('type', 'Unknown'),
        "Nullable": details.get('nullable', False),
        "Metadata": details.get('metadata', {})
    })

df = pd.DataFrame(schema_data)
df

,Column Name,Type,Nullable,Metadata
0,anchoring_prohibited,long,True,{}
1,removal_of_historic_artifacts_prohibited,long,True,{}
2,latest_updates,string,True,{}
3,managing_authority,string,True,{}
4,hook_n_line,long,True,{}
5,bottom_trawl,long,True,{}
6,nets,long,True,{}
7,site_id,string,True,{}
8,recreational_restrictions,long,True,{}
9,regulation_url,string,True,{}


### Filter by Country

In [8]:
filter_query = {
  "#EQUALS": [
    "$country",
    "Denmark"
  ]
}

In [9]:
# MPSdata = client.tabular.select_as_dataframe(dataset) # without filter
PSdata = client.tabular.select_as_dataframe(dataset, filter_query=filter_query) # with filter
PSdata

,regulation_url,boundary_source,inshore_only,definitions,anchoring_prohibited,construction_prohibited,hook_n_line,dredging_prohibited,trolling,tribal,...,site_name,coastline_match,latest_updates,industrial_or_mineral_exploration_prohibited,season,geometry_reduced,state,longlining,entry_prohibited,regulation_name
0,Fisheries Act|https://www.retsinformation.dk/F...,Flanders Marine Institute (2019). Maritime Bou...,0.0,None,3,3,3,3,3,1,...,Denmark EEZ (0-200NM),1.0,For updates on fisheries regulalations in mari...,3,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[16...",None,2,3,Fisheries Act; Executive Order on Recreational...
1,Act on Fishing and Hunting|http://extwprlegs1....,Flanders Marine Institute (2019). Maritime Bou...,0.0,None,3,3,3,3,3,1,...,Greenland Territorial Sea,1.0,None,3,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[-3...",None,3,3,Act on Fishing and Hunting (No. 12 of 1999); ...
2,Act on Fishing and Hunting|http://extwprlegs1....,Flanders Marine Institute (2019). Maritime Bou...,0.0,None,3,3,3,3,3,1,...,Greenland EEZ,1.0,None,3,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",None,3,3,Act on Fishing and Hunting (No. 12 of 1999); ...
3,Fisheries Act|https://www.retsinformation.dk/F...,Marine regions,0.0,None,3,3,3,3,3,1,...,Denmark Territorial Sea,1.0,For updates on fisheries regulalations in mari...,3,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[15...",None,2,3,Fisheries Act; Executive Order on Recreational...
4,Council Directive 92/43/EEC|https://eur-lex.eu...,Natura2000,0.0,None,3,3,3,3,3,1,...,"Vadehavet med Ribe Å, Tved Å og Varde Å vest f...",1.0,For updates on fisheries regulalations in mari...,3,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[8....",None,3,3,Council Directive 92/43/EEC of 21 May 1992 on ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,Council Directive 92/43/EEC|https://eur-lex.eu...,HELCOM,0.0,None,3,2,3,2,3,1,...,Hvideodde Rev,0.0,For updates on fisheries regulalations in mari...,2,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",None,3,3,Council Directive 92/43/EEC of 21 May 1992 on ...
225,Council Directive 92/43/EEC|https://eur-lex.eu...,HELCOM,0.0,None,3,2,2,2,3,1,...,Ebbeløkke Rev,0.0,For updates on fisheries regulalations in mari...,2,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[11...",None,1,3,Council Directive 92/43/EEC of 21 May 1992 on ...
226,Regulations Text|https://www.retsinformation.d...,HELCOM,0.0,None,3,3,3,3,3,1,...,Hesselø med omliggende stenrev,0.0,For updates on fisheries regulalations in mari...,3,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[11...",None,3,3,Decree on special fisheries regulation in mari...
227,HELCOM Recommendation|https://www.helcom.fi/Re...,HELCOM,0.0,None,3,2,3,2,3,1,...,Kims Top og den Kinesiske Mur,0.0,For updates on fisheries regulalations in mari...,2,Year-round,"{'type': 'MultiPolygon', 'coordinates': [[[[11...",None,3,3,HELCOM Recommendation 35/1; Council Directive ...


### Export Data in Different Formats

In [20]:
def export_PSdata(PSdata):
    """
    Export the PSdata DataFrame to selected file formats with browser downloads in Jupyter.
    
    Parameters:
    -----------
    PSdata : pandas.DataFrame or geopandas.GeoDataFrame
        The dataframe to be exported
    
    Returns:
    --------
    None
        Generates downloadable files for selected formats
    """
    # Available export formats
    export_formats = {
        'Shapefile': '.shp',
        'GeoJSON': '.geojson',
        'GPKG': '.gpkg',
        'CSV': '.csv',
        'SVG': '.svg'
    }
    
    # Create widgets
    format_label = widgets.Label("Select Export Formats:")
    format_checkboxes = {
        format_name: widgets.Checkbox(description=format_name, value=False) 
        for format_name in export_formats.keys()
    }
    
    # Download button and output
    download_button = widgets.Button(description="Generate Downloads")
    output = widgets.Output()
    
    # Arrange widgets
    format_box = widgets.VBox(list(format_checkboxes.values()))
    widget_layout = widgets.VBox([
        format_label, 
        format_box, 
        download_button, 
        output
    ])
    
    def create_download_link(data, filename):
        """
        Create a downloadable link for browser download.
        
        Parameters:
        -----------
        data : bytes
            File content to be downloaded
        filename : str
            Name of the file to be downloaded
        
        Returns:
        --------
        widgets.HTML
            A download link widget
        """
        b64 = base64.b64encode(data).decode()
        html = f'''
        <a download="{filename}" href="data:application/octet-stream;base64,{b64}" download>
            Download {filename}
        </a>
        '''
        return widgets.HTML(html)
    
    def on_download_click(b):
        with output:
            output.clear_output()
            
            # Determine selected formats
            selected_formats = [
                format_name for format_name, checkbox in format_checkboxes.items() 
                if checkbox.value
            ]
            
            if not selected_formats:
                print("❌ Error: No formats selected!")
                return
            
            # Download links to be displayed
            download_links = []
            
            # Export to selected formats
            try:
                for format_name in selected_formats:
                    # Prepare buffer for file content
                    buffer = io.BytesIO()
                    
                    # Export based on format
                    if format_name == 'Shapefile':
                        # Convert regular DataFrame to GeoDataFrame if needed
                        if not isinstance(PSdata, gpd.GeoDataFrame):
                            try:
                                # Attempt to convert to GeoDataFrame 
                                # Assumes you might have geometry column
                                gdf = gpd.GeoDataFrame(PSdata)
                            except Exception:
                                # Fallback: create a simple GeoDataFrame
                                gdf = gpd.GeoDataFrame(
                                    PSdata, 
                                    geometry=gpd.points_from_xy(
                                        PSdata.get('longitude', 0), 
                                        PSdata.get('latitude', 0)
                                    )
                                )
                        else:
                            gdf = PSdata
                        
                        # For Shapefile, we need to save multiple files
                        temp_dir = 'temp_shapefile'
                        os.makedirs(temp_dir, exist_ok=True)
                        shapefile_path = os.path.join(temp_dir, 'PSdata.shp')
                        gdf.to_file(shapefile_path)
                        
                        # Create a zip file of the shapefile components
                        import shutil
                        shutil.make_archive('PSdata_shapefile', 'zip', temp_dir)
                        
                        with open('PSdata_shapefile.zip', 'rb') as f:
                            buffer.write(f.read())
                        
                        # Clean up temporary files
                        shutil.rmtree(temp_dir)
                        os.remove('PSdata_shapefile.zip')
                        
                        download_link = create_download_link(
                            buffer.getvalue(), 
                            f'PSdata{export_formats[format_name]}.zip'
                        )
                    
                    elif format_name in ['GeoJSON', 'GPKG']:
                        # Convert regular DataFrame to GeoDataFrame if needed
                        if not isinstance(PSdata, gpd.GeoDataFrame):
                            try:
                                # Attempt to convert to GeoDataFrame 
                                # Assumes you might have geometry column
                                gdf = gpd.GeoDataFrame(PSdata)
                            except Exception:
                                # Fallback: create a simple GeoDataFrame
                                gdf = gpd.GeoDataFrame(
                                    PSdata, 
                                    geometry=gpd.points_from_xy(
                                        PSdata.get('longitude', 0), 
                                        PSdata.get('latitude', 0)
                                    )
                                )
                        else:
                            gdf = PSdata
                        
                        # Export to specific format
                        driver = 'GeoJSON' if format_name == 'GeoJSON' else 'GPKG'
                        gdf.to_file(buffer, driver=driver)
                        download_link = create_download_link(
                            buffer.getvalue(), 
                            f'PSdata{export_formats[format_name]}'
                        )
                    
                    elif format_name == 'CSV':
                        PSdata.to_csv(buffer, index=False)
                        download_link = create_download_link(
                            buffer.getvalue(), 
                            f'PSdata{export_formats[format_name]}'
                        )
                    
                    elif format_name == 'SVG':
                        import matplotlib.pyplot as plt
                        fig, ax = plt.subplots(figsize=(10, 10))
                        PSdata.plot(ax=ax)
                        plt.savefig(buffer, format='svg')
                        plt.close()
                        download_link = create_download_link(
                            buffer.getvalue(), 
                            f'PSdata{export_formats[format_name]}'
                        )
                    
                    download_links.append(download_link)
                
                # Display download links
                if download_links:
                    print("✅ Downloads generated!")
                    for link in download_links:
                        display(link)
                else:
                    print("❌ No valid downloads could be created.")
            
            except Exception as e:
                print(f"❌ Export Error: {e}")
    
    # Attach click event
    download_button.on_click(on_download_click)
    
    # Display widgets
    display(widget_layout)

In [21]:
export_PSdata(PSdata)